In [17]:
import numpy as np

In [18]:
edges = [(0, 1), (1, 2), (1, 3), (2, 3), (3, 0)]
input_nodes = [0]
output_nodes = [0]
node_nums = 4

input_value = 10
output_value = -5

In [19]:
def loop_circuit(input_nodes,output_nodes):
    xx,yy = np.meshgrid(output_nodes,input_nodes)
    result = np.stack((xx.ravel(),yy.ravel()),axis=1)
    return result.squeeze()

In [20]:
def is_in_array(node1,node2,loop_array):
    group = [node1,node2]
    if len(loop_array.shape) > 1:
        axis = 1
    else:
        axis = 0
    return np.any(np.all(loop_array == group,axis=axis))

In [21]:
def make_B(node_nums,edges,input_nodes,output_nodes):
    loop_nodes = loop_circuit(input_nodes,output_nodes)
    def get_vector(node1,node2):
        if node1 >= node2:
            return 1
        else:
            return -1
        
    P = np.zeros((node_nums))
    for i in range(P.size):
        if i in input_nodes:
            P[i] = input_value
        elif i in output_nodes:
            P[i] = output_value
        else:
            P[i] = 0

    P = P.reshape(-1,1)
    B = np.zeros((node_nums,len(edges)))

    for i in range(len(edges)):
        node1,node2 = edges[i]
        # if is_in_array(node1,node2,loop_nodes):
        #     B[node2,i] = -1
        #     B[node1,i] = 1
        B[node1,i] = 1
        B[node2,i] = -B[node1,i]
    return B

In [22]:
def make_H(edges,input_nodes,output_nodes):
    H = np.zeros((len(edges)))
    input_edges_index = [i[0] for i in enumerate(edges) for j in i[1] if j in input_nodes]
    output_edges_index = [i[0] for i in enumerate(edges) for j in i[1] if j in output_nodes]

    for i in range(len(edges)):
        if i in input_edges_index:
            H[i] += input_value
            if i in output_edges_index:
                H[i] -= output_value
        elif i in output_edges_index:
            H[i] -= output_value
        else:
            H[i] = 0

    return H.reshape(-1,1)

In [23]:
def Q_cal(B,P,H,R):
    result = np.sign(np.dot(B.T,P) + H)*np.sqrt(1/R * np.abs(np.dot(B.T,P) + H))
    return result

In [24]:
def xki(k,i,B,R,Q0,rho):
    temp = B[i,:].reshape(-1,1)*rho/np.where(R*np.abs(Q0)!=0,R*np.abs(Q0),1e-9)
    result = np.dot(B[k,:].reshape(-1,1).T,temp)
    return result.squeeze()

def X(B,R,Q0,rho):
    X = np.zeros((B.shape[0],B.shape[0]))
    for k in range(B.shape[0]):
        for i in range(B.shape[0]):
            X[k,i] = xki(k,i,B,R,Q0,rho)
    X = np.sum(X,axis=1,keepdims=True)
    X = np.where(X!=0,X,1e-9)
    return X

In [25]:
def Y(B,Q0,rho):
    Y = np.zeros((B.shape[0],1))
    for k in range(B.shape[0]):
        Y[k] = np.dot(B[k,:].reshape(-1,1).T,Q0*rho)
    return Y

In [26]:
B = make_B(node_nums,edges,input_nodes,output_nodes)

In [266]:
B = make_B(node_nums,edges,input_nodes,output_nodes)
P = np.zeros((node_nums,1))
H = make_H(edges,input_nodes,output_nodes)
R = np.ones((len(edges),1))
rho = np.ones((len(edges),1))

In [267]:
P[0] = 10
P[3] = -5
po = P

In [268]:
qo = Q_cal(B,po,H,R)
y = Y(B,qo,rho)
x = X(B,R,qo,rho)
delta_p = y/x

In [255]:
pn = P + delta_p
po = pn